In [1]:
from qiskit import QuantumCircuit
import qiskit.circuit as qc
from qiskit import QuantumRegister
from qiskit_ibm_provider import IBMProvider
from qiskit import transpile
import numpy as np
import math

provider = IBMProvider()
backend = provider.get_backend('ibm_perth')


In [31]:
backend.properties().gate_error('cx',[0,1])

0.018198344236854097

In [54]:
import random
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
from qiskit import Aer
from qiskit import transpile
def ising_1d_instance(qubits):
    def rand1d(qubits):
        return [random.choice([+1, -1]) for _ in range(qubits)]

    # transverse field terms
    h = rand1d(qubits)
    # links between lines
    j = rand1d(qubits-1)
    return (h, j)

nr_of_qubits = 4
h,j = ising_1d_instance(nr_of_qubits)
qr = QuantumRegister(nr_of_qubits)
circuit = QuantumCircuit(qr)
circuit.h([0])
circuit.h([1])
circuit.h([2])
circuit.h([3])
circuit.x(qubit=[0])
print(circuit)

def add_measurement(circuit, qubits):
    # Create a Quantum Circuit
    meas = QuantumCircuit(qubits, qubits)
    meas.barrier(range(qubits))
    # map the quantum measurement to the classical bits
    meas.measure(range(qubits), range(qubits))

    # The Qiskit circuit object supports composition using
    # the compose method.
    circuit.add_register(meas.cregs[0])
    qc = circuit.compose(meas)
    return qc

qc = add_measurement(circuit, nr_of_qubits)
print(qc)

backend_sim = Aer.get_backend('qasm_simulator')
counts = backend_sim.run(transpile(qc, backend_sim), shots=1024).result().get_counts()

print(counts)

        ┌───┐┌───┐
q489_0: ┤ H ├┤ X ├
        ├───┤└───┘
q489_1: ┤ H ├─────
        ├───┤     
q489_2: ┤ H ├─────
        ├───┤     
q489_3: ┤ H ├─────
        └───┘     
        ┌───┐┌───┐ ░ ┌─┐         
q489_0: ┤ H ├┤ X ├─░─┤M├─────────
        ├───┤└───┘ ░ └╥┘┌─┐      
q489_1: ┤ H ├──────░──╫─┤M├──────
        ├───┤      ░  ║ └╥┘┌─┐   
q489_2: ┤ H ├──────░──╫──╫─┤M├───
        ├───┤      ░  ║  ║ └╥┘┌─┐
q489_3: ┤ H ├──────░──╫──╫──╫─┤M├
        └───┘      ░  ║  ║  ║ └╥┘
   c: 4/══════════════╩══╩══╩══╩═
                      0  1  2  3 
{'1010': 84, '0001': 60, '0110': 64, '1011': 68, '0010': 63, '0000': 65, '1100': 58, '1110': 60, '0101': 53, '1101': 64, '1000': 53, '1111': 71, '0100': 59, '0111': 72, '1001': 68, '0011': 62}


In [71]:
shots = 1024

def compute_expected_energy(counts,h,j,shots):
    def bool_to_state(integer):
        # Convert the 1/0 of a bit to +1/-1
        return 2*int(integer)-1

    # Get total energy of each count
    r1=list(counts.keys())
    r2=list(counts.values())
    total_energy = 0

    for k in range(0,len(r1)):
        # r2[k] is the number of shots that have this result
        # r1[k] is the result as qubits (like 0001)
        
        # Energy of h
        for bit_h in range(0,len(r1[k])):
            total_energy += bool_to_state(r1[k][bit_h]) * h[bit_h]*r2[k]
        # Energy of j
        for bit_value in range(0,len(r1[k])-1):
            total_energy += bool_to_state(r1[k][bit_value])*bool_to_state(r1[k][bit_value+1])*j[bit_value]*r2[k]
    expectation_value = total_energy/shots
    return expectation_value

# 0001 -> -1,-1,-1,1
# ij_1 -1*-1*j[0]



# def digit_sum(n):
#     num_str = str(n)
#     sum = 0
#     for i in range(0, len(num_str)):
#         sum += int(num_str[i])
#     return sum
def bool_to_state(integer):
    # Convert the 1/0 of a bit to +1/-1
    return 2*int(integer)-1
r1=list(counts.keys())
r2=list(counts.values())
h = [-1,1,1,-1]
print(r1)
print(r2)
total = 0
for k in range(0,len(r1)):
    print(total)
    total += sum([bool_to_state(r1[k][bit_value])*h[bit_value] for bit_value in range(0,len(r1[k]))])*r2[k]
# for k in range(0,len(r1)):
#         expectation_value+=(4-2*digit_sum(r1[j]))*r2[j]/shots
# print(expectation_value)

['1010', '0001', '0110', '1011', '0010', '0000', '1100', '1110', '0101', '1101', '1000', '1111', '0100', '0111', '1001', '0011']
[84, 60, 64, 68, 63, 65, 58, 60, 53, 64, 53, 71, 59, 72, 68, 62]
0
0
-120
136
0
126
126
126
246
246
118
12
12
130
274
2


In [83]:
from qiskit.circuit import QuantumCircuit, Parameter
import numpy as np
phi = Parameter("1")
phi2 = Parameter("2")
qr = QuantumRegister(nr_of_qubits)
circuit = QuantumCircuit(qr)
circuit.h([0])
circuit.h([1])
circuit.h([2])
circuit.h([3])
circuit.rx(phi,qubit=[0])
circuit.rx(phi2,qubit=[1])
print(circuit)
print(circuit.parameters)
value = np.random.random()
parameters = [value for _ in range(2)]
print(circuit.bind_parameters(parameters))

       ┌───┐┌───────┐
q68_0: ┤ H ├┤ Rx(1) ├
       ├───┤├───────┤
q68_1: ┤ H ├┤ Rx(2) ├
       ├───┤└───────┘
q68_2: ┤ H ├─────────
       ├───┤         
q68_3: ┤ H ├─────────
       └───┘         
ParameterView([Parameter(1), Parameter(2)])
       ┌───┐┌───────────────────────┐
q68_0: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤├───────────────────────┤
q68_1: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤└───────────────────────┘
q68_2: ┤ H ├─────────────────────────
       ├───┤                         
q68_3: ┤ H ├─────────────────────────
       └───┘                         


In [84]:
from CLQGA import genome_to_circuit, compute_gradient

In [105]:
# TEST for gradient computation
CIRCUIT_DEPTH = 6
GATE_ENCODING_LENGTH = 6 
QUBIT_SECTIONING_LENGTH = 5
NR_OF_QUBITS = 5
NR_OF_GATES = CIRCUIT_DEPTH
NR_OF_ISING_QUBITS = 4
CHROMOSOME_LENGTH = CIRCUIT_DEPTH * (GATE_ENCODING_LENGTH + QUBIT_SECTIONING_LENGTH)
genome = ''.join(str(random.randint(0,1)) for _ in range(CHROMOSOME_LENGTH))
print(genome)
circuit, nr_of_parameters = genome_to_circuit(genome, NR_OF_QUBITS, NR_OF_GATES)
print(circuit)
observable_h, observable_j = ising_1d_instance(NR_OF_ISING_QUBITS)
gradient, circuit = compute_gradient(circuit, nr_of_parameters, NR_OF_ISING_QUBITS, observable_h, observable_j, 1024)
print(gradient)
print(circuit)

101111010110000111011001100100010111011000110100100001011101101110
                    
q206_0: ────────────
        ┌──────────┐
q206_1: ┤ U(0,1,2) ├
        └──────────┘
q206_2: ────────────
                    
q206_3: ────────────
                    
q206_4: ────────────
                    
{'0000': 117, '0010': 907}
total_energy =  -4418
EXPECTED ENERGY =  -4.314453125
{'0010': 932, '0000': 92}
total_energy =  -4568
EXPECTED ENERGY =  -4.4609375
{'0000': 104, '0010': 920}
total_energy =  -4496
EXPECTED ENERGY =  -4.390625
{'0000': 99, '0010': 925}
total_energy =  -4526
EXPECTED ENERGY =  -4.419921875
{'0000': 108, '0010': 916}
total_energy =  -4472
EXPECTED ENERGY =  -4.3671875
{'0000': 94, '0010': 930}
total_energy =  -4556
EXPECTED ENERGY =  -4.44921875
257.8125
                                                                 
q206_0: ─────────────────────────────────────────────────────────
        ┌───────────────────────────────────────────────────────┐
q206_1: ┤ U(3.786418

In [6]:
from qiskit.providers.ibmq import IBMQ
from qiskit.providers.fake_provider import FakeProviderForBackendV2
from qiskit.providers import aer
from HelperCLQGA import find_backend
backend = "ibm_perth"
# provider = FakeProviderForBackendV2()
# available_cloud_backends = provider.backends()
# print(available_cloud_backends)
# for i in available_cloud_backends: 
#     if i.name == backend:
#         backend = i
provider = FakeProviderForBackendV2()
available_cloud_backends = provider.backends()

In [8]:
for i in available_cloud_backends: 
     print(i.name)
#  backend_sim.properties().gate_error('cx',[0,1])

fake_almaden_v2
fake_armonk_v2
fake_athens_v2
fake_auckland
fake_belem_v2
fake_boeblingen_v2
fake_bogota_v2
fake_brooklyn_v2
fake_burlington_v2
fake_cairo_v2
fake_cambridge_v2
fake_casablanca_v2
fake_essex_v2
fake_geneva
fake_guadalupe_v2
fake_hanoi_v2
fake_jakarta_v2
fake_johannesburg_v2
fake_kolkata_v2
fake_lagos_v2
fake_lima
fake_london_v2
fake_manhattan_v2
fake_manila_v2
fake_melbourne_v2
fake_montreal_v2
fake_mumbai_v2
fake_nairobi_v2
fake_oslo
fake_ourense_v2
fake_paris_v2
fake_perth
fake_prague
fake_poughkeepsie_v2
fake_quito_v2
fake_rochester_v2
fake_rome_v2
fake_santiago_v2
fake_sherbrooke
fake_singapore_v2
fake_sydney_v2
fake_toronto_v2
fake_valencia_v2
fake_vigo_v2
fake_washington_v2
fake_yorktown_v2


In [11]:
import random
chromosome_child_0 = ""
chromosome_child_1 = ""
parents = ["aaaaa","bbbbb"]
for i in range(0,len(parents[0])):
    if random.randint(0,1):
        chromosome_child_0 +=parents[1][i]
        chromosome_child_1 +=parents[0][i]
    else:
        chromosome_child_0 += parents[0][i]
        chromosome_child_1 += parents[1][i]

print("1, ",chromosome_child_0)
print("2, ",chromosome_child_1)

1,  aaaab
2,  bbbba


In [23]:
individual = "0000000"
elitism_population = ["1110111","1111111"]
match_percentage = []
def match(subject_1,subject_2):
    return sum([int(subject_1[i] == subject_2[i]) for i in range(0,len(subject_1))])/len(subject_1)
for elite in elitism_population:
    match_percentage.append(match(individual,elite))
    print(match_percentage)
print(sum(match_percentage)/len(match_percentage))

[0.14285714285714285]
[0.14285714285714285, 0.0]
0.07142857142857142
